<img src='https://user-images.githubusercontent.com/16665629/36117115-329c6220-1041-11e8-98d3-7cd1ce05503c.jpeg'
style="float:right;margin:0 10px 0px 0;width: 150px;">

# Compare observed and theoretical statistical distributions

In [1]:
models_compare = {
                    'skymodel1.txt': 'pybdsm-meerkat_SourceRecovery-lwimager.lsm.html',
                    'skymodel2.txt': 'pybdsm-meerkat_SourceRecovery-wsclean.lsm.html',
                    'skymodel3.txt': 'pybdsm-meerkat_SourceRecovery-ddfacet.lsm.html',
                    'skymodel4.txt': 'pybdsm-meerkat_SourceRecovery-tclean.lsm.html',
                 }
models_dr = {
            'pybdsm-meerkat_SourceRecovery-lwimager.lsm.html': 'meerkat_SourceRecovery_lwimager-cube.residual.fits',
            'pybdsm-meerkat_SourceRecovery-wsclean.lsm.html': 'meerkat_SourceRecovery_wsclean-cube.residual.fits',
            'pybdsm-meerkat_SourceRecovery-ddfacet.lsm.html': 'meerkat_SourceRecovery_ddfacet-cube.residual.fits',
            'pybdsm-meerkat_SourceRecovery-tclean.lsm.html': 'meerkat_SourceRecovery_tclean-cube.residual.fits',
            }
RESIDUALS = [
             'meerkat_SourceRecovery_lwimager-cube.residual.fits',
             'meerkat_SourceRecovery_wsclean-cube.residual.fits',
             'meerkat_SourceRecovery_ddfacet-cube.residual.fits',
             'meerkat_SourceRecovery_tclean-cube.residual.fits',
            ]
res_noise_images = {'model': 'skymodel1.txt',
                    'meerkat_SourceRecovery_lwimager-cube.residual.fits': 'lwimager_noise.dirty.fits',
                    'meerkat_SourceRecovery_wsclean-cube.residual.fits': 'wsclean_noise-MFS-image.fits',
                    'meerkat_SourceRecovery_ddfacet-cube.residual.fits': 'ddfacet_noise.app.restored.fits',
                    'meerkat_SourceRecovery_tclean-cube.residual.fits': 'tclean_noise.dirty.image.fits',
                    }
tolerance = 0.00001 # area factor around the source

In [2]:
from ipywidgets import widgets, interact
from IPython.display import Javascript, display
from IPython.display import HTML

def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+5)'))

directory = widgets.Text(
    value='input',
    placeholder='input',
    description='Input directory:',
    disabled=False
)

button = widgets.Button(description="Configure")
button.on_click(run_all)
display(button)
display(directory)

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} $( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit"
value="Click here to toggle on/off the raw code."></form>''')

Button(description=u'Configure', style=ButtonStyle())

Text(value=u'input', description=u'Input directory:', placeholder=u'input')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
import glob
import combat
import matplotlib
import numpy as np
from plotly import tools
import plotly.graph_objs as go
from plotly import offline as py
from collections import OrderedDict
from scipy.stats import norm, linregress
from sklearn.metrics import r2_score, mean_squared_error
from plotly.graph_objs import Layout, Figure, XAxis, YAxis
from aimfast.aimfast import residual_image_stats, image_dynamic_range, model_dynamic_range
py.init_notebook_mode(connected=True)

In [38]:
reload(combat)
ARCSEC_DEG = 1.0/(60*60)
results = combat.property_results(models=models_compare, tolerance=tolerance, input=directory.value)

Loading input/output/skymodel1.txt: ASCII table
Loading input/output/pybdsm-meerkat_SourceRecovery-lwimager.lsm.html: Tigger sky model
Number of sources recovered: 84
Loading input/output/skymodel4.txt: ASCII table
Loading input/output/pybdsm-meerkat_SourceRecovery-tclean.lsm.html: Tigger sky model


1.3Gb lsmformats(ASCII.py:258:load): Set default freq0 to 1420000000.0 from source on line 2.
1.3Gb lsmformats(ASCII.py:258:load): Set default freq0 to 1420000000.0 from source on line 2.


Number of sources recovered: 78
Loading input/output/skymodel3.txt: ASCII table
Loading input/output/pybdsm-meerkat_SourceRecovery-ddfacet.lsm.html: Tigger sky model
Number of sources recovered: 86
Loading input/output/skymodel2.txt: ASCII table
Loading input/output/pybdsm-meerkat_SourceRecovery-wsclean.lsm.html: Tigger sky model


1.3Gb lsmformats(ASCII.py:258:load): Set default freq0 to 1420000000.0 from source on line 2.
1.3Gb lsmformats(ASCII.py:258:load): Set default freq0 to 1420000000.0 from source on line 2.


Number of sources recovered: 89


In [39]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+27)'))

button = widgets.Button(
    description="PLOT"
)
button.on_click(run_all)
_scaley = widgets.Checkbox(
    value=False,
    description='Same Y-scale',
    disabled=False
)
_scalex = widgets.Checkbox(
    value=False,
    description='Same X-scale',
    disabled=False
)
data_range = widgets.IntSlider(
    value=100,
    min=1,
    max=100,
    step=1,
    description='Bin by flux(%):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [40]:
display(button)
display(_scaley)
display(_scalex)
display(data_range)

Button(description=u'PLOT', style=ButtonStyle())

Checkbox(value=False, description=u'Same Y-scale')

Checkbox(value=False, description=u'Same X-scale')

IntSlider(value=100, continuous_update=False, description=u'Bin by flux(%):', min=1)

<IPython.core.display.Javascript object>

In [41]:
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))

## Photometric Measurements

In [42]:
bg_color = 'rgb(229,229,229)'
bg_color = ''
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
for input_model, output_model in models_compare.items():
    counter+=1
    name_labels = []
    flux_in_data = []
    flux_out_data = []
    source_scale = []
    phase_center_dist = []
    flux_out_err_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        flux_out_data.append(results[heading]['flux'][n][0])
        flux_out_err_data.append(results[heading]['flux'][n][1])
        flux_in_data.append(results[heading]['flux'][n][2])
        name_labels.append(results[heading]['flux'][n][3])
        phase_center_dist.append(results[heading]['position'][n][-3])
        source_scale.append(results[heading]['shape'][n][3])
    zipped_props = zip(flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale)
    flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    I_min_max += flux_out_data
    err_I_min_max += flux_out_err_data
    flux_R_score = r2_score(flux_in_data, flux_out_data)
    flux_MSE = mean_squared_error(flux_in_data, flux_out_data)
    reg = linregress(flux_in_data, flux_out_data)
    I_out_in = [float(I_out)/I_in for I_out,I_in in zip(flux_out_data,flux_in_data)]
    print("Model: {:s} \n{:s} \nSlope: {:.4f} | Mean Squared Error: {:.2E} | R2: {:.4f} "
          "Intercept: {:.4f}\n{:s}".format(heading, '-'*45, reg.slope, flux_MSE, flux_R_score,
                                           reg.intercept, '='*75))
    counter += 1
matplotlib.pylab.close()

Model: pybdsm-meerkat_SourceRecovery-lwimager 
--------------------------------------------- 
Slope: 0.9906 | Mean Squared Error: 4.16E-09 | R2: 0.9854 Intercept: -0.0000
Model: pybdsm-meerkat_SourceRecovery-tclean 
--------------------------------------------- 
Slope: 0.8974 | Mean Squared Error: 1.00E-07 | R2: 0.6169 Intercept: -0.0001
Model: pybdsm-meerkat_SourceRecovery-ddfacet 
--------------------------------------------- 
Slope: 0.9178 | Mean Squared Error: 1.97E-08 | R2: 0.9338 Intercept: -0.0000
Model: pybdsm-meerkat_SourceRecovery-wsclean 
--------------------------------------------- 
Slope: 0.9086 | Mean Squared Error: 1.94E-08 | R2: 0.9395 Intercept: 0.0000


In [43]:
data = []
im_titles =[]
flux_im = dict()
for input_model, output_model in models_compare.items():
    header = output_model[:-9]
    im_titles.append('<b>{:s} flux density</b>'.format(header.upper()))

fig = tools.make_subplots(rows=len(models_compare.keys()), cols=1, shared_yaxes=False, print_grid=False,
                          vertical_spacing = 0.06, subplot_titles=im_titles)
j = 0
i = -1
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
annotate = []
for input_model, output_model in models_compare.items():
    i += 1
    counter += 1
    name_labels = []
    flux_in_data = []
    flux_out_data = []
    source_scale = []
    phase_center_dist = []
    flux_out_err_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        flux_out_data.append(results[heading]['flux'][n][0])
        flux_out_err_data.append(results[heading]['flux'][n][1])
        flux_in_data.append(results[heading]['flux'][n][2])
        name_labels.append(results[heading]['flux'][n][3])
        phase_center_dist.append(results[heading]['position'][n][-3])
        source_scale.append(results[heading]['shape'][n][3])
    zipped_props = zip(flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale)
    flux_out_data, flux_out_err_data, flux_in_data, name_labels, phase_center_dist, source_scale = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    flux_R_score = r2_score(flux_in_data, flux_out_data)
    flux_MSE = mean_squared_error(flux_in_data, flux_out_data)
    reg = linregress(flux_in_data, flux_out_data)
    I_out_in = [float(I_out)/I_in for I_out,I_in in zip(flux_out_data,flux_in_data)]
    annotate.append(go.Annotation(
            x=0.0014,
            y=flux_in_data[-1] + 0.0001,
            xref='x{:d}'.format(counter),
            yref='y{:d}'.format(counter),
            text="Slope: {:.4f} | Intercept: {:.4f} | Mean Squared Error: {:.2E} | R2: {:.4f} ".format(
                reg.slope, reg.intercept, flux_MSE, flux_R_score),
            ax=0,
            ay=-40,
            showarrow=False,
            bordercolor='#c7c7c7',
            borderwidth=2,
            font=dict(color = "black", size = 10.2),
        ))
    fig.append_trace(go.Scatter(x=np.array([flux_in_data[0], flux_in_data[-1]]), showlegend=False,
                                marker = dict(color = 'rgb(0,0,255)'),
                                y=np.array([flux_in_data[0], flux_in_data[-1]]), mode = 'line'), i+1, 1)
    fig.append_trace(go.Scatter(x=np.array(flux_in_data), y=np.array(flux_out_data),
                                mode = 'markers', showlegend=False,
                                text = name_labels, name = '%s flux_ratio' % heading,
                                marker = dict(color = phase_center_dist, showscale=True, colorscale='Jet',
                                              reversescale=False,
                                              colorbar = dict(title='Distance from phase center (arcsec)',
                                                              titleside ='right', len=0.2, y=0.9-j)
                                             ),
                                error_y=dict(type='data', array=flux_out_err_data,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig['layout'].update(title='', height=2000, width=600,
                     #    paper_bgcolor='rgb(255,255,255)', plot_bgcolor=bg_color,
                         legend=dict(x=0.8,y=1.0),)
    fig['layout'].update(
        {'yaxis{}'.format(counter):YAxis(title=u'$I_{out} (Jy)$',gridcolor='rgb(255,255,255)',
        #range=[1,10],
        tickfont=dict(size=15),
        titlefont=dict(size=17),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter):XAxis(title=u'$I_{in} (Jy)$', #position=0.5,
                                                            titlefont=dict(size=17),
                                                            showgrid=True,
                                                            showline=True,
                                                            overlaying='x')})
    if counter == len(models_compare.keys()):
        fig['layout']['annotations'].extend(annotate)
    j+=0.27
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Astrometric Measurements

In [44]:
counter = 0
pos_min_max = []
DEC_off_min_max = []
RA_off_min_max = []
for input_model, output_model in models_compare.items():
    counter+=1
    RA_offset = []
    DEC_offset = []
    DELTA_PHASE0 = []
    source_labels = []
    flux_in_data = []
    delta_pos_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        delta_pos_data.append(results[heading]['position'][n][0])
        RA_offset.append(results[heading]['position'][n][1])
        DEC_offset.append(results[heading]['position'][n][2])
        DELTA_PHASE0.append(results[heading]['position'][n][3])
        flux_in_data.append(results[heading]['position'][n][4])
        source_labels.append(results[heading]['position'][n][5])
    zipped_props = zip(delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels)
    delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    pi,sin,cos = np.pi,np.sin,np.cos
    r1, r2 = np.array(RA_offset).std(), np.array(DEC_offset).std()
    theta = np.linspace(0,2*pi,45)
    x1 = r1*cos(theta)
    y1 = r2*sin(theta)
    print("{:s}\n1 sigma = {:f}\n{:s}".format(heading, np.sqrt(r1*r2), 20*'='))

pybdsm-meerkat_SourceRecovery-lwimager
1 sigma = 0.109982
pybdsm-meerkat_SourceRecovery-tclean
1 sigma = 0.280090
pybdsm-meerkat_SourceRecovery-ddfacet
1 sigma = 0.086783
pybdsm-meerkat_SourceRecovery-wsclean
1 sigma = 0.068843


In [45]:
data = []
im_titles =[]
for input_model, output_model in models_compare.items():
    header = output_model[:-9]
#    im_titles.append('<b>{:s} Position</b>'.format(header.upper()))
#    im_titles.append('<b>{:s} Position</b>'.format(header.upper()))

fig = tools.make_subplots(rows=len(models_compare.keys()), cols=2, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.25,
                          vertical_spacing = 0.15,
                          subplot_titles=im_titles)
j = 0
i = -1
counter = 0
for input_model, output_model in models_compare.items():
    i += 1
    counter+=1
    RA_offset = []
    DEC_offset = []
    DELTA_PHASE0 = []
    source_labels = []
    flux_in_data = []
    delta_pos_data = []
    heading = output_model[:-9]
    for n in range(len(results[heading]['flux'])):
        delta_pos_data.append(results[heading]['position'][n][0])
        RA_offset.append(results[heading]['position'][n][1])
        DEC_offset.append(results[heading]['position'][n][2])
        DELTA_PHASE0.append(results[heading]['position'][n][3])
        flux_in_data.append(results[heading]['position'][n][4])
        source_labels.append(results[heading]['position'][n][5])
    zipped_props = zip(delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels)
    delta_pos_data, RA_offset, DEC_offset, DELTA_PHASE0, flux_in_data, source_labels = zip(
        *sorted(zipped_props, key=lambda x: x[-2]))
    fig.append_trace(go.Scatter(x=np.array(flux_in_data), y=np.array(delta_pos_data),
                                mode = 'markers', showlegend=False,
                                text = source_labels, name = '{:s} flux_ratio'.format(header),
                                marker = dict(color = DELTA_PHASE0, showscale=True,
                                              colorscale='Jet', reversescale=True,
                                              colorbar = dict(title='Distance from phase center (arcsec)',
                                                              titleside ='right', len=0.2, y=0.93-j, x=0.39)
                                             ),
                                error_y=dict(type='data', #array=flux_out_err_data,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=np.array(RA_offset), y=np.array(DEC_offset),
                                mode = 'markers', showlegend=False,
                                text = source_labels, name = '{:s} flux_ratio'.format(heading),
                                marker = dict(color = flux_out_data, showscale=True, colorscale='Viridis',
                                              reversescale=True,
                                              colorbar = dict(title='Output flux (Jy)',
                                                              titleside ='right', len=0.2, y=0.93-j)
                                             ),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 2)
    r1, r2 = np.array(RA_offset).std(), np.array(DEC_offset).std()
    theta = np.linspace(0,2*pi,45)
    x1 = r1*cos(theta)
    y1 = r2*sin(theta)
    fig.append_trace(go.Scatter(x=x1, y=y1,
                                mode = 'lines', showlegend=True if i == 0 else False,
                                name = '1σ',
                                text = '1σ ~ {:f}'.format(np.sqrt(r1*r2)),
                                marker = dict(color = 'rgb(0, 0, 255)')), i+1, 2)
    fig.append_trace(go.Scatter(x=2*x1, y=2*y1,
                                mode = 'lines', showlegend=True if i == 0 else False,
                                name = '2σ',
                                text = '2σ ~ {:f}'.format(2*np.sqrt(r1*r2)),
                                marker = dict(color = 'rgb(255, 0, 0)')), i+1, 2)
    fig.append_trace(go.Scatter(x=3*x1, y=3*y1,
                                mode = 'lines', showlegend=True if i == 0 else False,
                                name = '3σ',
                                text = '3σ ~ {:f}'.format(3*np.sqrt(r1*r2)),
                                marker = dict(color = 'rgb(255, 255, 0)')), i+1, 2)
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', plot_bgcolor=bg_color,
                         legend=dict(xanchor=True, x=1.2,y=1)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'Delta position [arcsec]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
                                           range=axis_min_max if _scaley.value else [],
        #range=[1,10],
        tickfont=dict(size=14, color='rgb(0,0,0)'),
        titlefont=dict(size=15),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=True)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title='Declination offset [arcsec]',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
                                             range=dec_min_max if _scaley.value else [],
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=17),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=True)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='Input Flux (Jy)',
                                                            titlefont=dict(size=17),
                                                            zeroline=True, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Right Ascension offset [arcsec]',
                                                              titlefont=dict(size=17),
                                                              range=ra_min_max if _scalex.value else [],
                                                              zeroline=False)})# domain=[0.505, 0.8])})
    fig['layout']['annotations'].update({ 'font':{'size': 10}})
    j+=0.28
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Morphological Measurements

In [46]:
data = []
im_titles =[]
for input_model, output_model in models_compare.items():
    header = output_model[:-9]
#    im_titles.append('<b>{:s} Source shape</b>'.format(header.upper()))
#    im_titles.append('<b>{:s} Source shape</b>'.format(header.upper()))

fig = tools.make_subplots(rows=len(models_compare.keys()), cols=2, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.25,
                          vertical_spacing = 0.15,
                          subplot_titles=im_titles)
j = 0
i = -1
counter = 0
for input_model, output_model in models_compare.items():
    i += 1
    counter+=1
    SCALE = []
    SCALE_ERR = []
    flux_in_data = []
    DELTA_PHASE0 = []
    source_labels = []
    MAJ_MIN_angle_in = []
    MAJ_MIN_angle_out = []
    heading = output_model[:-9]
    for n in range(len(results[header]['flux'])):
        MAJ_MIN_angle_out.append(results[header]['shape'][n][0])
        MAJ_MIN_angle_in.append(results[header]['shape'][n][2])
        SCALE.append(results[header]['shape'][n][3])
        SCALE_ERR.append(results[header]['shape'][n][4])
        flux_in_data.append(results[header]['shape'][n][5])
        source_labels.append(results[header]['shape'][n][6])
        DELTA_PHASE0.append(results[header]['position'][n][-3])  
    zipped_props = zip(MAJ_MIN_angle_out, MAJ_MIN_angle_in, DELTA_PHASE0, SCALE, SCALE_ERR)
    MAJ_MIN_angle_out, MAJ_MIN_angle_in, DELTA_PHASE0, SCALE, SCALE_ERR = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    maj_in = [maj_min_angle_in[0] for maj_min_angle_in in MAJ_MIN_angle_in]
    maj_out = [maj_min_angle_out[0] for maj_min_angle_out in MAJ_MIN_angle_out]
    min_in = [maj_min_angle_in[1] for maj_min_angle_in in MAJ_MIN_angle_in]
    min_out = [maj_min_angle_out[1] for maj_min_angle_out in MAJ_MIN_angle_out]
    angle_offset = [(maj_min_angle_out[2] - maj_min_angle_in[2]) for maj_min_angle_out, maj_min_angle_in in zip(
        MAJ_MIN_angle_out, MAJ_MIN_angle_in)]
    fig.append_trace(go.Scatter(x=maj_in, y=maj_out, mode = 'markers', showlegend=False,
                                text = name_labels, name = '{:s} flux_ratio'.format(header),
                                marker = dict(color = DELTA_PHASE0, showscale=True, colorscale='Jet',
                                              reversescale=False,
                                              colorbar = dict(title='Distance from phase center (arcsec)',
                                                              titleside ='right', len=0.2, y=0.93-j, x=0.39)
                                             ),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=min_in, y=min_out,
                                mode = 'markers', showlegend=False,
                                text = name_labels, name = '{:s} flux_ratio'.format(header),
                                marker = dict(color = DELTA_PHASE0, showscale=True,
                                              colorscale='Jet', reversescale=False,
                                              colorbar = dict(title='Distance from phase center (arcsec)',
                                                              titleside ='right', len=0.2, y=0.93-j)
                                             ),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 2)
    pi,sin,cos = np.pi,np.sin,np.cos
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', plot_bgcolor=bg_color,
                         legend=dict(xanchor=True)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'Output major axis[arcsec]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=14),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=True)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title='Ouput minor axis[arcsec]',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=14),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=True)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='Input major axis[arcsec]',
                                                            titlefont=dict(size=14),
                                                            zeroline=False, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Input minor axis[arcsec]',
                                                              titlefont=dict(size=14),
                                                              zeroline=False)})# domain=[0.505, 0.8])}
    fig['layout']['annotations'].update({'font':{'size': 12}})
    j+=0.28
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Spectral Index Measurements

In [47]:
data = []
im_titles =[]
flux_im = dict()
for input_model, output_model in models_compare.items():
    header = output_model[:-9]
#    im_titles.append('<b>{:s} spi measure</b>'.format(header.upper()))

fig = tools.make_subplots(rows=len(models_compare.keys()), cols=1, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.005, vertical_spacing = 0.15,
                          subplot_titles=im_titles)
i = -1
j = 0
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []
if results[header]['spi']:
    for input_model, output_model in models_compare.items():
        i += 1
        counter+=1
        I_in = []
        name_labels = []
        spi_in_data = []
        spi_out_data = []
        spi_err_data = []
        dist_from_phase = []
        for n in range(len(results[header]['spi'])):
            spi_out_data.append(results[header]['spi'][n][0])
            spi_err_data.append(results[header]['spi'][n][1])
            spi_in_data.append(results[header]['spi'][n][2])
            dist_from_phase.append(results[header]['spi'][n][3])
            I_in.append(results[header]['spi'][n][4])
            name_labels.append(results[header]['spi'][n][5])
        zipped_props = zip(spi_out_data, spi_err_data, spi_in_data, dist_from_phase, I_in, name_labels)
        (spi_out_data, spi_err_data, spi_in_data, dist_from_phase, I_in, name_labels) = zip(
            *sorted(zipped_props, key=lambda x: x[0]))
        spi_R_score = r2_score(spi_in_data, spi_out_data)
        spi_MSE = mean_squared_error(spi_in_data, spi_out_data)
        spi_out_in = [float(spi_out)/spi_in for spi_out,spi_in in zip(spi_out_data,spi_in_data)]
    #    fig.append_trace(go.Scatter(x=np.array([spi_out_data[0], spi_out_data[-1]]), showlegend=False,
    #                                y=np.array([spi_out_data[0], spi_out_data[-1]]), mode = 'line'), i+1, 1)
        fig.append_trace(go.Scatter(x=np.array([sorted(I_in)[0], sorted(I_in)[-1]]), showlegend=False,
                                    marker = dict(color = 'rgb(0,0,255)'),
                                    y=np.array([-.7,-.7]), mode = 'line'), i+1, 1)
        fig.append_trace(go.Scatter(x=np.array(I_in), y=np.array(spi_out_data),
                                    mode = 'markers', showlegend=False,
                                    text = name_labels, name = '%s flux_ratio' % heading,
                                    marker = dict(color = phase_center_dist, showscale=True, colorscale='Jet',
                                                  reversescale=False,
                                                  colorbar = dict(title='Distance from phase center (arcsec)',
                                                                  titleside ='right', len=0.2, y=0.93-j)
                                                 ),
                                    error_y=dict(type='data', array=spi_err_data,
                                                 color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
        fig['layout'].update(title='', height=900, width=900,
                             paper_bgcolor='rgb(255,255,255)', plot_bgcolor=bg_color,
                             legend=dict(x=0.8,y=1.0),)
        fig['layout'].update(
            {'yaxis{}'.format(counter):YAxis(title=u'$SPI_{out}$',gridcolor='rgb(255,255,255)',
            #range=[1,10],
            tickfont=dict(size=15),
            titlefont=dict(size=17),
            showgrid=True,
            showline=True,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=True)})
        fig['layout'].update({'xaxis{}'.format(counter):XAxis(title='$I_{in} (Jy)$', position=0.0,
                                                                titlefont=dict(size=17),
                                                                overlaying='x')})
        fig['layout']['annotations'].update({ 'font':{'size': 10}})
        j +=0.28
    py.iplot(fig, filename='make-subplots-multiple-with-titles')

## Dynamic Ranges

In [48]:
beam_factor = 2.0
DRs = OrderedDict()
im_titles = []
dir = directory.value
for m, r in models_dr.items():
    DRs[r[:-19]] = model_dynamic_range('{:s}/{:s}'.format(dir, m),'{:s}/{:s}'.format(dir, r), 5.5, beam_factor)
#    DRs[r[:-19]] = model_dynamic_range('{:s}/{:s}'.format(dir, m),'{:s}/{:s}'.format(dir, r), 94, beam_factor)
zipped_props = zip(DRs.keys(), DRs.values())
ims, values = zip(*sorted(zipped_props, key=lambda x: x[0]))
drs, names = [], []
for val in values:
    drs.append(val[0])
    names.append("DR=({:f},{:f})".format(val[1], val[-1]))
fig = tools.make_subplots(rows=1, cols=1, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.005, vertical_spacing = 0.15)
fig.append_trace(go.Scatter(x=ims, showlegend=False, text = names,
                            y=np.array(drs), mode = 'markers', marker=dict(size='30')), 1, 1)
#fig['layout'].update(title='<b>SNR with self-cal iterations</b>', height=900, width=900,
fig['layout'].update(title='', height=900, width=900,
                     paper_bgcolor='rgb(255,255,255)', #plot_bgcolor='rgb(229,229,229)',
                     legend=dict(x=0.8,y=1.0))
fig['layout'].update(
    {'yaxis{}'.format(1):YAxis(title=u'Dynamic Range',gridcolor='rgb(255,255,255)',
        tickfont=dict(size=15),
        titlefont=dict(size=17),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
fig['layout'].update({'xaxis{}'.format(1):XAxis(title='iterations', position=0.0,
                                                        titlefont=dict(size=17),
                                                        showline=True,
                                                        overlaying='x')})
fig['layout']['titlefont'].update({'size':20})
py.iplot(fig, filename='make-subplots-multiple-with-titles')

Loading input/output/pybdsm-meerkat_SourceRecovery-wsclean.lsm.html: Tigger sky model
Loading input/output/pybdsm-meerkat_SourceRecovery-lwimager.lsm.html: Tigger sky model
Loading input/output/pybdsm-meerkat_SourceRecovery-tclean.lsm.html: Tigger sky model
Loading input/output/pybdsm-meerkat_SourceRecovery-ddfacet.lsm.html: Tigger sky model


## Residual Statistics

In [49]:
Res = OrderedDict()
for res in RESIDUALS:
    Res[res] = residual_image_stats('{:s}/{:s}'.format(directory.value, res))

im_titles = [
               'MEAN',
               'STANDARD DEVIATION',
               'SKEWNESS',
               'KURTOSIS'
            ]
    
fig = tools.make_subplots(rows=4, cols=1, shared_yaxes=False, print_grid=False,
                          horizontal_spacing = 0.005, vertical_spacing = 0.15,)
                    #      subplot_titles=im_titles)
    
names = []
mean = []
std = []
skew = []
kurt = []
for name, stats in Res.items():
    names.append(name[:-19])
    mean.append(stats['MEAN'])
    std.append(stats['STDDev'])
    skew.append(stats['SKEW'])
    kurt.append(stats['KURT'])
zipped_props = zip(names, mean, std)
names, mean, std = zip(*sorted(zipped_props, key=lambda x: x[0]))
x_axis_title = 'Imagers'
#x_axis_title = 'Iterations'
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(mean), mode = 'marker'), 1, 1)
fig['layout'].update(title='', height=900, width=900,
                     paper_bgcolor='rgb(255,255,255)', #plot_bgcolor='rgb(229,229,229)',
                         legend=dict(x=0.8,y=1.0),)
fig['layout'].update(
    {'yaxis{}'.format(1):YAxis(title=u'Residual mean (Jy)',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=17),
            showgrid=True,
            showline=True,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(1):XAxis(title=x_axis_title, position=0.0,
                                                titlefont=dict(size=17),
                                                showline=True,
                                                overlaying='x')})
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(std), mode = 'marker'), 2, 1)
fig['layout'].update(
    {'yaxis{}'.format(2):YAxis(title=u'Residual sigma (Jy)',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=17),
            showgrid=True,
            showline=True,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(2):XAxis(title=x_axis_title, position=0.0,
                                                titlefont=dict(size=17),
                                                showline=True,
                                                overlaying='x')})
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(skew)*1000, mode = 'marker'), 3, 1)
fig['layout'].update(
    {'yaxis{}'.format(3):YAxis(title=u'Residual skewness (mJy)',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=17),
            showgrid=True,
            showline=True,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            dtick='',
            tickformat='',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(3):XAxis(title=x_axis_title, position=0.0,
                                                titlefont=dict(size=17),
                                                showline=True,
                                                overlaying='x')})
fig.append_trace(go.Scatter(x=names, showlegend=False,
                            y=np.array(kurt), mode = 'marker'), 4, 1)
fig['layout'].update(
    {'yaxis{}'.format(4):YAxis(title=u'Residual kurtosis',gridcolor='rgb(255,255,255)',
            tickfont=dict(size=15),
            titlefont=dict(size=17),
            showgrid=True,
            showline=True,
            showticklabels=True,
            tickcolor='rgb(51,153,225)',
            ticks='outside',
            zeroline=False)})
fig['layout'].update({'xaxis{}'.format(4):XAxis(title=x_axis_title, position=0.0,
                                                titlefont=dict(size=17),
                                                showline=True,
                                                overlaying='x')})
py.iplot(fig, filename='make-subplots-multiple-with-titles')

#### Source residual-to-noise ration measurements

In [50]:
reload(combat)
try:
    skymodel = res_noise_images.pop('model')
except KeyError:
    pass
res = combat.res_noise_ratio(skymodel, res_noise_images, directory=directory.value)

Loading input/output/skymodel1.txt: ASCII table


1.3Gb lsmformats(ASCII.py:258:load): Set default freq0 to 1420000000.0 from source on line 2.


In [51]:
data = []
im_titles =[]
TO_MICRO = 1E6

for residual_image, noise_image in res_noise_images.items():
    header1 = residual_image[:-5][23:]
    header2 = noise_image[:-17][:]
    im_titles.append('<b>{:s} RMS</b>'.format(header1.upper()))
    im_titles.append('<b>{:s} residual-noise</b>'.format(header2.upper()))

fig = tools.make_subplots(rows=len(res_noise_images), cols=2, shared_yaxes=False, print_grid=False,
                          #horizontal_spacing = 0.25,
                          vertical_spacing = 0.15,
                          subplot_titles=im_titles)
counter = 0
i=0
j=0
for residual_image, noise_image in res_noise_images.items():
    counter+=1
    SNR = []
    dist_from_phase = []
    src_list = []
    rmss = []
    residuals = []
    res_noise_ratio = []
    res_mean_noise_rms_ratio = []
    imager = header = residual_image
    k = 0
    for res_src in res[imager]:
        src_list.append(res_src[0])
        residuals.append(res_src[2])
        rmss.append(res_src[3])
        res_noise_ratio.append(res_src[4])
        res_mean_noise_rms_ratio.append(res_src[8])
        dist_from_phase.append(res_src[-1])
        k += 1
#    for n in range(len(results[imager]['spi'])):
#        SCALE.append(results[imager]['shape'][n][4])
    fig.append_trace(go.Scatter(x=range(len(src_list)), y=np.array(rmss)*TO_MICRO, mode = 'lines',
                                showlegend=True if i == 0 else False,
                                text = name_labels, name = 'noise',
                                marker = dict(color = 'rgb(255,0,0)'),
                                error_y=dict(type='data', #array=SCALE_ERR,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=range(len(src_list)), y=np.array(residuals)*TO_MICRO, mode = 'lines',
                                showlegend=True if i == 0 else False,
                                text = name_labels, name = 'residual',
                                marker = dict(color = 'rgb(0,0,255)'),
                                error_y=dict(type='data', #array=SCALE_ERR,
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=range(len(src_list)), y=np.array(res_noise_ratio),
                                mode = 'markers', showlegend=False,
                                text = name_labels,# name = '%s flux_ratio' % imager,
                                marker = dict(color = dist_from_phase, showscale=True, colorscale='Jet',
                                              reversescale=False,
                                              colorbar = dict(title='Distance from phase center (arcsec)',
                                                              titleside ='right', len=0.2, y=0.93-j, x=0.99)
                                             ),
                                error_y=dict(type='data',
                                             color = 'rgb(158, 63, 221)', visible=True)), i+1, 2)
    pi,sin,cos = np.pi,np.sin,np.cos
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', #plot_bgcolor='rgb(229,229,229)',
                         legend=dict(xanchor=True, x=1.12,y=1)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'rms [μJy]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
        #range=[1,10],
        tickfont=dict(size=14, color='rgb(0,0,0)'),
        titlefont=dict(size=17),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title='residual_flux/noise',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
                                             range=res_axis_min_max if _scaley.value else [],
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=15),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='Source',
                                                            titlefont=dict(size=17),
                                                            showline=True,
                                                            zeroline=False, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Source',
                                                              titlefont=dict(size=17),
                                                              showline=True,
                                                              zeroline=False)}# domain=[0.505, 0.8])}
        )
    i+=1
    j+=0.285
py.iplot(fig, filename='make-subplots-multiple-with-titles')

#### Random source residual-to-noise ration measurements

In [52]:
reload(combat)
try:
    skymodel = res_noise_images.pop('model')
except KeyError:
    pass
res = combat.random_res_noise_ratio(res_noise_images, directory=directory.value,
                                    num_pix=4096, pix_size=1.0, num_areas=200)

In [53]:
data = []
im_titles =[]
TO_MICRO = 1E6

for residual_image, noise_image in res_noise_images.items():
    header1 = residual_image[:-5][23:]
    header2 = noise_image[:-17][:]
    im_titles.append('<b>{:s} RMS</b>'.format(header1.upper()))
    im_titles.append('<b>{:s} residual-noise</b>'.format(header2.upper()))

fig = tools.make_subplots(rows=len(res_noise_images), cols=2, shared_yaxes=False, print_grid=False,
                          #horizontal_spacing = 0.25,
                          vertical_spacing = 0.15,
                          subplot_titles=im_titles)
i=0
j=0
counter = 0
for residual_image, noise_image in res_noise_images.items():
    counter+=1
    rmss = []
    residuals = []
    res_noise_ratio = []
    imager = residual_image
    k = 0
    for res_src in res[imager]:
        residuals.append(res_src[0])
        rmss.append(res_src[1])
        res_noise_ratio.append(res_src[2])
        k += 1
    fig.append_trace(go.Scatter(x=range(len(rmss)), y=np.array(rmss)*TO_MICRO, mode = 'lines',
                                showlegend=True if i == 0 else False,
                                text = name_labels, name = 'noise',
                                marker = dict(color = 'rgb(255,0,0)'),
                                error_y=dict(type='data', #array=SCALE_ERR,
                                             color='rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=range(len(rmss)), y=np.array(residuals)*TO_MICRO, mode = 'lines',
                                showlegend=True if i == 0 else False,
                                text = name_labels, name = 'residual',
                                marker = dict(color='rgb(0,0,255)'),
                                error_y=dict(type='data', #array=SCALE_ERR,
                                             color='rgb(158, 63, 221)', visible=True)), i+1, 1)
    fig.append_trace(go.Scatter(x=range(len(rmss)), y=np.array(res_noise_ratio),
                                mode = 'markers', showlegend=False,
                                text = name_labels,# name = '%s flux_ratio' % imager,
                                 marker = dict(color='rgb(158, 63, 221)'),
                                error_y=dict(type='data',
                                             color='rgb(158, 63, 221)', visible=True)), i+1, 2)
    pi,sin,cos = np.pi,np.sin,np.cos
    fig['layout'].update(title='', height=900, width=900,
                         paper_bgcolor='rgb(255,255,255)', #plot_bgcolor='rgb(229,229,229)',
                         legend=dict(xanchor=True, x=1.0,y=1.05)
                        )
    fig['layout'].update(
        {'yaxis{}'.format(counter+i):YAxis(title=u'rms [μJy]',gridcolor='rgb(255,255,255)',
                                           color='rgb(0,0,0)',
        #range=[1,10],
        tickfont=dict(size=14, color='rgb(0,0,0)'),
        titlefont=dict(size=17),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update(
        {'yaxis{}'.format(counter+i+1):YAxis(title='residual_flux/noise',gridcolor='rgb(255,255,255)',
                                             color='rgb(0,0,0)',
                                             range=res_axis_min_max if _scaley.value else [],
        #range=[1,10],
        tickfont=dict(size=10, color='rgb(0,0,0)'),
        titlefont=dict(size=15),
        showgrid=True,
        showline=True,
        showticklabels=True,
        tickcolor='rgb(51,153,225)',
        ticks='outside',
        zeroline=False)})
    fig['layout'].update({'xaxis{}'.format(counter+i):XAxis(title='Source',
                                                            titlefont=dict(size=17),
                                                            showline=True,
                                                            zeroline=False, position=0.0, overlaying='x',)})
    fig['layout'].update({'xaxis{}'.format(counter+i+1):XAxis(title='Source',
                                                              titlefont=dict(size=17),
                                                              showline=True,
                                                              zeroline=False)}# domain=[0.505, 0.8])}
        )
    i+=1
    j+=0.285
py.iplot(fig, filename='make-subplots-multiple-with-titles')

## View Reconstructed Maps

In [54]:
%%html
<iframe src="http://js9.si.edu/js9/js9.html" width=100% height=1000></iframe>

## Source meta-data

In [55]:
src_nm = widgets.Text(
    value='',
    placeholder='ay040',
    description='Source Name:',
    disabled=False
)
display(src_nm)

Text(value=u'', description=u'Source Name:', placeholder=u'ay040')

In [56]:
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+2)'))

button = widgets.Button(description="Search")
button.on_click(run_all)
display(button)

Button(description=u'Search', style=ButtonStyle())